In [1]:
import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import GRU
from timeit import default_timer as timer

Using TensorFlow backend.


In [2]:
print(keras.__version__)
print(tf.__version__)

2.2.4
1.12.0


#### 加载数据，打乱数据，切分数据，整理数据维度

In [3]:
# 路径
train_fn_0 = '../../dataset/train_0.csv'
train_fn_1 = '../../dataset/train_1.csv'
test_fn_0 = '../../dataset/test_0.csv'
test_fn_1 = '../../dataset/test_1.csv'
# 加载训练集
train_0 = np.loadtxt(train_fn_0,dtype=np.float32,delimiter = ',',encoding='utf-8-sig')
train_1 = np.loadtxt(train_fn_1,dtype=np.float32,delimiter = ',',encoding='utf-8-sig')
train = np.concatenate((train_0,train_1),axis=0)
np.random.shuffle(train)
# 加载测试集
test_0 = np.loadtxt(test_fn_0,dtype=np.float32,delimiter = ',',encoding='utf-8-sig')
test_1 = np.loadtxt(test_fn_1,dtype=np.float32,delimiter = ',',encoding='utf-8-sig')
test = np.concatenate((test_0, test_1), axis=0)
np.random.shuffle(test)
# 整理数据维度
x_train = train[:,0:30][:,:,np.newaxis]
x_test = test[:,0:30][:,:,np.newaxis]
y_train = train[:,30][:,np.newaxis]
y_test = test[:,30][:,np.newaxis]
print(x_train.shape)
print(y_train.shape)

(38022, 30, 1)
(38022, 1)


In [4]:
print(x_test.shape)
print(y_test.shape)

(9505, 30, 1)
(9505, 1)


#### GRU

In [9]:
GRU_model = Sequential()
GRU_model.add(GRU(units=64, activation='relu', return_sequences=True, input_shape=(30, 1)))
GRU_model.add(Dropout(0.2))
GRU_model.add(GRU(units=32, activation='relu', return_sequences=True))
GRU_model.add(Dropout(0.2))
GRU_model.add(GRU(units=16, activation='relu', return_sequences=False))
GRU_model.add(Dense(1, activation='sigmoid'))

GRU_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
GRU_model.summary()
start_time = timer()
GRU_model.fit(x_train, y_train, validation_split=0.3,batch_size=512, epochs=10)
current_time = timer()
print("train_time",(current_time-start_time)/10)

score = GRU_model.evaluate(x_test, y_test, batch_size=512)
print(score)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_3 (GRU)                  (None, 30, 64)            12672     
_________________________________________________________________
dropout_3 (Dropout)          (None, 30, 64)            0         
_________________________________________________________________
gru_4 (GRU)                  (None, 30, 32)            9312      
_________________________________________________________________
dropout_4 (Dropout)          (None, 30, 32)            0         
_________________________________________________________________
gru_5 (GRU)                  (None, 16)                2352      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 24,353
Trainable params: 24,353
Non-trainable params: 0
_________________________________________________________________
Train 

In [10]:
from sklearn.metrics import classification_report
gru_y_pred=GRU_model.predict(x_test, batch_size=512)
print(classification_report(y_test, gru_y_pred.round(),digits=4))

              precision    recall  f1-score   support

         0.0     0.9909    0.9838    0.9874      6872
         1.0     0.9586    0.9765    0.9675      2633

   micro avg     0.9818    0.9818    0.9818      9505
   macro avg     0.9748    0.9802    0.9774      9505
weighted avg     0.9820    0.9818    0.9819      9505



In [11]:
# 计算模型预测时间
test_time_gru = []
for i in range(10):
    start_time = timer()
    GRU_model.predict(x_test, batch_size=500)
    current_time = timer()
    test_time_gru.append(current_time-start_time)
    print(current_time-start_time)
    i = i+1
print(test_time_gru)
print(np.mean(test_time_gru))

0.7344348430633545
0.6896342635154724
0.7183783799409866
0.6988086923956871
0.7291321158409119
0.714407280087471
0.713363341987133
0.7017052546143532
0.705484427511692
0.7048412039875984
[0.7344348430633545, 0.6896342635154724, 0.7183783799409866, 0.6988086923956871, 0.7291321158409119, 0.714407280087471, 0.713363341987133, 0.7017052546143532, 0.705484427511692, 0.7048412039875984]
0.711018980294466
